# Лабораторная работа 1. Введение в машинное обучение. Обучение с учителем. Задача регрессии

<b>Традиционное предупреждение для всех лабораторных работ:</b> перед обучением моделей необходимо выполнить предварительную обработку данных, которая <b>обязательно</b> включает в себя:
- заполнение пропущенных значений (рекомедуется логика заполнения пропусков на основе типа данных, которая использовалась в РГР по Практикуму);
- преобразование категориальных признаков в числовые (используйте one-hot кодирование или map; используйте знания с Практикума).

Предобработка может включать в себя другие действия, но выполнение описанных выше действий обязательно.

Сделайте это один раз и сохраните в отдельный csv файл, а потом его используйте.

<b>Выполните следующие задания:</b>
- загрузите датасет для регрессии, выделите целевой признак и предикторы, разбейте данные на обучающую и тестовую выборку;
- решите задачу регрессии на ваших данных с использованием моделей sklearn (линейная регрессия + L1, L2), для моделей с регуляризациями подберите гиперпараметр;
- решите задачу регрессии на ваших данных с использованием моделей sklearn (полиномиальная регрессия + L1, L2), для моделей с регуляризациями подберите гиперпараметр;
- вычислите значения метрик $R^2$, MAE, MSE, RMSE, MAPE для всех обученных моделей; выберите лучшую модель;
- самостоятельно реализуйте (желательно в виде класса) модель линейной регрессии с регуляризацией (можете выбрать L1 или L2);
- самостоятельно реализуйте вычисление всех используемых метрик (в виде функций, принимающих два аргумента);
- обучите вашу модель линейной регрессии на ваших данных; оцените качество с помощью реализованных вами метрик.

In [1]:
from pathlib import Path
import os
import sys

sys.path.append(str(Path(os.getcwd()).parent))

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
from math import sqrt
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import numpy as np
from sklearn.preprocessing import PolynomialFeatures

Функция для вычисления метрик

In [3]:
def metrics(test, predict):
    print(f'MAE: {mean_absolute_error(test, predict)}')
    print(f'MSE: {mean_squared_error(test, predict)}')
    print(f'RMSE: {sqrt(mean_squared_error(test, predict))}')
    print(f'MAPE: {mean_absolute_percentage_error(test, predict)}')
    print(f'R^2: {r2_score(test, predict)}')

Функция для заполнения пропусков в выборке

In [4]:
def fill_empty_cell(column_name, df):
    if df.dtypes[column_name] == "float64":
        df[column_name] = df[column_name].fillna(df[column_name].mean())
    elif df.dtypes[column_name] == "int64":
        df[column_name] = df[column_name].fillna(df[column_name].median())
    else:
        df[column_name] = df[column_name].fillna(df[column_name].mode())

Ridge

In [5]:
def create_ridge_with_hyperparameter(X_train, y_train) -> Ridge:
    alpha = {'alpha': np.arange(0, 1, 0.1)}
    ridge = GridSearchCV(Ridge(), alpha).fit(X_train, y_train)
    return ridge

Lasso

In [6]:
def create_lasso_with_hyperparameter(X_train, y_train) -> Lasso:
    alpha = {'alpha': np.arange(0, 1, 0.1)}
    lasso = RandomizedSearchCV(Lasso(), alpha).fit(X_train, y_train)
    return lasso

In [7]:
table = pd.read_csv("../data/energy_task.csv", parse_dates=['date'])

In [8]:
table['month'] = pd.DatetimeIndex(table['date']).month
table.drop(['date'], axis=1, inplace=True)
table

,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,...,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,month
0,60,30,19.890000,47.596667,19.200000,44.790000,19.790000,44.730000,19.000000,45.566667,...,48.900000,17.033333,45.5300,6.60,733.5,92.000000,7.000000,63.000000,5.3,11.0
1,60,30,19.890000,46.693333,19.200000,44.722500,19.790000,44.790000,19.000000,45.992500,...,48.863333,17.066667,45.5600,6.48,733.6,92.000000,6.666667,59.166667,5.2,11.0
2,50,30,19.890000,46.300000,19.200000,44.626667,19.790000,44.933333,18.926667,45.890000,...,48.730000,17.000000,45.5000,6.37,733.7,92.000000,6.333333,55.333333,5.1,11.0
3,50,40,19.890000,46.066667,19.200000,44.590000,19.790000,45.000000,18.890000,45.723333,...,48.590000,17.000000,45.4000,6.25,733.8,92.000000,6.000000,51.500000,5.0,11.0
4,60,40,19.890000,46.333333,19.200000,44.530000,19.790000,45.000000,18.890000,45.530000,...,48.590000,17.000000,45.4000,6.13,733.9,92.000000,5.666667,47.666667,4.9,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19730,100,0,25.566667,46.560000,25.890000,42.025714,27.200000,41.163333,24.700000,45.590000,...,50.074000,23.200000,46.7900,22.70,755.2,55.666667,3.333333,23.666667,13.3,5.0
19731,90,0,25.500000,46.500000,25.754000,42.080000,27.133333,41.223333,24.700000,45.590000,...,49.790000,23.200000,46.7900,22.60,755.2,56.000000,3.500000,24.500000,13.3,5.0
19732,270,10,25.500000,46.596667,25.628571,42.768571,27.050000,41.690000,24.700000,45.730000,...,49.660000,23.200000,46.7900,22.50,755.2,56.333333,3.666667,25.333333,13.3,5.0
19733,420,10,25.500000,46.990000,25.414000,43.036000,26.890000,41.290000,24.700000,45.790000,...,49.518750,23.200000,46.8175,22.30,755.2,56.666667,3.833333,26.166667,13.2,5.0


Проверка датасета на наличие пустых данных

In [9]:
null_columns = []
for i in table.columns:
    if len(table[table[i].isnull()]) > 0:
        null_columns.append(i)
print(null_columns)

['RH_2', 'RH_4', 'RH_5', 'RH_6', 'T7', 'T8', 'T9', 'RH_9', 'T_out', 'Press_mm_hg', 'RH_out', 'Visibility', 'month']


In [10]:
for name in null_columns:
    fill_empty_cell(name, table)

In [11]:
for i in table.columns:
    if len(table[table[i].isnull()]) > 0:
        print(f"В столбце {i} есть пустые элементы")

In [12]:
table.to_csv('../data/energy_task_compleated.csv') 

Создание обучающей выборки

In [13]:
y = table["Appliances"]
X = table.drop(["Appliances"], axis=1)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

1. Простая линейная регрессия

In [15]:
lr = LinearRegression().fit(X_train, y_train)
y1_predict = lr.predict(X_test)
metrics(y_test, y1_predict)

MAE: 53.10679567708878
MSE: 8591.182844508243
RMSE: 92.68863384745858
MAPE: 0.6258338469268974
R^2: 0.16611682728825938


In [16]:
from mylib.LinearRegressionClass import My_LinearRegression 


linear = My_LinearRegression(X_train, y_train).fit()
y_my = linear.predict(X_test)

In [17]:
from mylib.Metrics import all_metrics


all_metrics(y_test, y_my)

MAE: 57.38854942076182
MSE: 9701.246411417216
RMSE: 98.49490550996643
MAPE: 0.7081670039971508
R^2: 0.05837109007845487


In [18]:
from mylib.Metrics import all_metrics


all_metrics(y_test, y1_predict)

MAE: 53.10679567708878
MSE: 8591.182844508243
RMSE: 92.68863384745858
MAPE: 0.6258338469268974
R^2: 0.16611682728825938


2. L1 - Lasso (подбор гиперпараметра + модель)

In [19]:
l = create_lasso_with_hyperparameter(X_train, y_train)
y2_predict = l.predict(X_test)
metrics(y_test, y2_predict)

c:\Users\bugae\MachineLearning-Labs\venv\lib\site-packages\sklearn\model_selection\_validation.py:686: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
c:\Users\bugae\MachineLearning-Labs\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
c:\Users\bugae\MachineLearning-Labs\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.731e+07, tolerance: 1.144e+04 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklear

MAE: 53.07905846640824
MSE: 8591.912619663875
RMSE: 92.69257046637489
MAPE: 0.625245261270526
R^2: 0.16604599336082937


3. L2 - Ridge (подбор гиперпараметра + модель)

In [20]:
r = create_ridge_with_hyperparameter(X_train, y_train)
y3_predict = r.predict(X_test)
metrics(y_test, y3_predict)

MAE: 53.10667473422766
MSE: 8591.199699996327
RMSE: 92.68872477273774
MAPE: 0.6258313873079989
R^2: 0.16611519124952812


4. Полиномиальная регрессия

In [21]:
table_p = table.drop(columns=["T1", "RH_4", "RH_6", "T7"])
y_p = table_p["Appliances"]
X_p = table_p.drop(["Appliances"], axis=1)
X_p_train, X_p_test, y_p_train, y_p_test = train_test_split(X_p, y_p, test_size=0.5, random_state=42)

In [22]:
p = PolynomialFeatures(2)
X_PolynomialFeatures_train = p.fit_transform(X_p_train)
X_PolynomialFeatures_test = p.fit_transform(X_p_test)

In [23]:
lr2 = LinearRegression().fit(X_PolynomialFeatures_train, y_p_train)
y_PolynomialFeatures_predict = lr2.predict(X_PolynomialFeatures_test)
metrics(y_p_test, y_PolynomialFeatures_predict)

TypeError: LinearRegression.__init__() missing 2 required positional arguments: 'X_train' and 'y_train'

5. Полиномиальная регрессия + Ridge

In [ ]:
r_p = create_ridge_with_hyperparameter(X_PolynomialFeatures_train, y_p_train)
y_p_ridge_predict = r_p.predict(X_PolynomialFeatures_test)
metrics(y_p_test, y_p_ridge_predict)

MAE: 51.028233135692595
MSE: 8318.029152423323
RMSE: 91.20322994512487
MAPE: 0.5709522699518464
R^2: 0.22716292020549944


6. Полиномиальная регрессия + Lasso

In [ ]:
l_p = create_lasso_with_hyperparameter(X_PolynomialFeatures_train, y_p_train)
y_p_lasso_predict = l_p.predict(X_PolynomialFeatures_test)
metrics(y_p_test, y_p_ridge_predict)

c:\Users\bugae\MachineLearning-Labs\venv\lib\site-packages\sklearn\model_selection\_validation.py:686: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
c:\Users\bugae\MachineLearning-Labs\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
c:\Users\bugae\MachineLearning-Labs\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.055e+07, tolerance: 8.158e+03 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklear

MAE: 51.028233135692595
MSE: 8318.029152423323
RMSE: 91.20322994512487
MAPE: 0.5709522699518464
R^2: 0.22716292020549944


c:\Users\bugae\MachineLearning-Labs\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.783e+07, tolerance: 1.012e+04 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
